In [145]:
import json as json
import pathlib as pl
from copy import deepcopy
from datetime import datetime
from gc import collect
from os import path, system

import matplotlib.pyplot as plt
from matplotlib import rcParams
from numpy import array
from numpy.linalg import norm
from tqdm import tqdm

from calc_catenary_from_ext_points import get_coor_marker_points_ideal_catenary
from utils import check, gif_from_pngs, print_dict

gain = 1.5
rcParams[ 'figure.figsize' ] = [ gain * 6.4, gain * 4.8 / 2 ]

In [163]:
for v in pl.Path( "./export" ).glob( '*' ):
	print( str( v ).split( '//' )[ -1 ], end = '\t' )
	print( datetime.fromtimestamp( path.getctime( v ) ) )

export\three_robots_chain_1726157070	2024-09-17 09:04:31.536003
export\three_robots_chain_1726514783	2024-09-17 09:04:31.562367
export\three_robots_chain_1726580899	2024-09-17 15:48:19.796810
export\three_robots_chain_with_fixed_end_1726429003	2024-09-17 09:04:31.589695
export\three_robots_chain_with_fixed_end_1727078633	2024-09-23 10:03:53.983169
export\three_robots_chain_with_fixed_end_1727081819	2024-09-23 10:56:59.165230
export\three_robots_chain_with_fixed_end_1727099624	2024-09-23 15:53:44.695520


In [184]:
folder = pl.Path( './export/three_robots_chain_with_fixed_end_1727099624' )
uuid = str( folder ).split( '_' )[ -1 ]

In [194]:
with open( folder / 'config.json' ) as f:
	config = json.load( f )
print_dict( config )

model:
	dynamics: dynamics
	step: step
	model_dynamics:
		state_size: 48
		actuation_size: 18
		br0_pose: 0:6:None
		br0_position: 0:3:None
		br0_xy: 0:2:None
		br0_z: 2
		br0_orientation: 3:6:None
		br1_pose: 6:12:None
		br1_position: 6:9:None
		br1_xy: 6:8:None
		br1_z: 8
		br1_orientation: 9:12:None
		br2_pose: 12:18:None
		br2_position: 12:15:None
		br2_xy: 12:14:None
		br2_z: 14
		br2_orientation: 15:18:None
		br0_speed: 24:30:None
		br0_linear_speed: 24:27:None
		br0_angular_speed: 27:30:None
		br1_speed: 30:36:None
		br1_linear_speed: 30:33:None
		br1_angular_speed: 33:36:None
		br2_speed: 36:42:None
		br2_linear_speed: 36:39:None
		br2_angular_speed: 39:42:None
		br0_actuation_start: 0
		br0_actuation: 0:6:None
		br0_linear_actuation: 0:3:None
		br0_angular_actuation: 3:6:None
		br1_actuation_start: 6
		br1_actuation: 6:12:None
		br1_linear_actuation: 6:9:None
		br1_angular_actuation: 9:12:None
		br2_actuation_start: 12
		br2_actuation: 12:18:None
		br2_linear_actuation: 12:15:

In [195]:
slice_repository = { }
axes = [ 'x', 'y', 'z', 'phi', 'theta', 'psi', 'u', 'v', 'w', 'p', 'q', 'r' ]

for k, v in config[ 'model' ][ 'model_dynamics' ].items():
	if isinstance( v, str ) and ':' in v:
		slice_repository[ k ] = slice( *[ int( v ) if v != 'None' else None for v in v.split( ':' ) ] )

positions = { k: v for k, v in slice_repository.items() if 'position' in k }
orientations = { k: v for k, v in slice_repository.items() if 'orientation' in k }
positions_pairs = { a.split( '_' )[ 0 ]: b.split( '_' )[ 0 ] for a, b in
										zip( list( positions )[ :-1 ], list( positions )[ 1: ] ) }

In [196]:
for i, c in enumerate( config[ 'constraints' ] ):
	print( f'Constraint {i}' )
	for j, l in enumerate( c[ 'labels' ] ):
		print( f'\t{j}: {l}' )

Constraint 0
	0: $z_0+H_{01}$
	1: $z_1+H_{12}$
	2: $z_2+H_{2fe}$
	3: $|P_0^{x,y}-P_1^{x,y}|$
	4: $|P_1^{x,y}-P_2^{x,y}|$
	5: $|P_2^{x,y}-P_fe^{x,y}|$
	6: $|P_0^{x,y,z}-P_1^{x,y,z}|$
	7: $|P_1^{x,y,z}-P_2^{x,y,z}|$
	8: $|P_2^{x,y,z}-P_fe^{x,y,z}|$
	9: $|V_0|$
	10: $|V_1|$
	11: $|V_2|$


In [197]:
data_files = list( folder.glob( 'data/*' ) )
data_files.sort( key = lambda x: path.getmtime( x ) )
with open( data_files[ -1 ] ) as f:
	final_state = json.load( f )

times = [ config[ 'model' ][ 'time_step' ] * i for i in range( len( config[ 'target_trajectory' ] ) ) ]
target_trajectory = array( config[ 'target_trajectory' ] )[ :, 0 ]

n_frames = len( data_files )
previous_times = [ config[ 'model' ][ 'time_step' ] * i for i in range( n_frames ) ]
previous_target_trajectory = target_trajectory[ :n_frames ]
previous_actual_trajectory = array( final_state[ 'model' ][ 'previous_states' ] )[ 1:,
														 :config[ 'model' ][ 'model_dynamics' ][ 'state_size' ] // 2 ]

pose_weight = array( config[ 'pose_weight_matrix' ][ 0 ] )

In [152]:
check( f'{folder}/plots' )

11

In [198]:
for k, v in positions.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	absolute_distances = abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] )
	plt.plot( previous_times, absolute_distances )
	plt.legend(
			[
					r'over $\mathbf{x}_w$-axis',
					r'over $\mathbf{y}_w$-axis',
					r'over $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [199]:
for k, v in orientations.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	plt.plot( previous_times, abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] ) )
	plt.legend(
			[
					r'around $\mathbf{x}_w$-axis',
					r'around $\mathbf{y}_w$-axis',
					r'around $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [rad]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [200]:
for p1, p2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, positions[ p1 + '_position' ] ] -
			previous_actual_trajectory[ :, positions[ p2 + '_position' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [201]:
for p1, p2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, slice_repository[ p1 + '_xy' ] ] -
			previous_actual_trajectory[ :, slice_repository[ p2 + '_xy' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/horizontal_distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [202]:
figs = [ 'x', 'y', 'z' ]

x = plt.figure().subplots()
y = plt.figure().subplots()
z = plt.figure().subplots()
z.invert_yaxis()

legends = [ ]

for k, v in positions.items():
	position = previous_actual_trajectory[ :, v ]
	X = position[ :, 0 ]
	Y = position[ :, 1 ]
	Z = position[ :, 2 ]
	x.plot( previous_times, X )
	y.plot( previous_times, Y )
	z.plot( previous_times, Z )
	legends += [ f'{k}' ]
	if norm( pose_weight[ v, v ] ) > 0:
		target = previous_target_trajectory[ :, v ]
		Xt = target[ :, 0 ]
		Yt = target[ :, 1 ]
		Zt = target[ :, 2 ]
		x.plot( previous_times, Xt, ':', linewidth = 3 )
		y.plot( previous_times, Yt, ':', linewidth = 3 )
		z.plot( previous_times, Zt, ':', linewidth = 3 )
		legends += [ f'{k} target' ]

for f in plt.get_fignums():
	fig = plt.figure( f )
	plt.legend( legends )
	plt.xlabel( 'time [s]' )
	plt.ylabel( f'position on $\mathbf{{{figs[ f - 1 ]}}}_w$-axis [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/positions_on_{figs[ f - 1 ]}_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [158]:
frames_already_done = check( f'{folder}/plots/3d_frames' )

In [159]:
for frame in tqdm( range( frames_already_done, n_frames ) ):
	# frame = 1000

	fig = plt.figure( figsize = (10, 10) )
	view = plt.subplot( projection = '3d' )
	view.set_xlabel( "$\mathbf{x}_w$-axis" )
	view.set_ylabel( "$\mathbf{y}_w$-axis" )
	view.set_zlabel( "$\mathbf{z}_w$-axis" )

	inset_view_xz = view.inset_axes( [ .0, .0, .2, .2 ] )
	inset_view_xz.set_xlabel( "$\mathbf{x}_w$-axis" )
	inset_view_xz.set_ylabel( "$\mathbf{z}_w$-axis" )
	inset_view_xz.invert_yaxis()

	inset_view_yz = view.inset_axes( [ .8, .0, .2, .2 ] )
	inset_view_yz.set_xlabel( "$\mathbf{y}_w$-axis" )
	inset_view_yz.set_ylabel( "$\mathbf{z}_w$-axis" )
	inset_view_yz.invert_xaxis()
	inset_view_yz.invert_yaxis()

	inset_view_xy = view.inset_axes( [ .0, .8, .2, .2 ] )
	inset_view_xy.set_xlabel( "$\mathbf{x}_w$-axis" )
	inset_view_xy.set_ylabel( "$\mathbf{y}_w$-axis" )
	inset_view_xy.invert_yaxis()

	for k, v in positions.items():
		if norm( pose_weight[ v, v ] ) > 0:
			Xt = target_trajectory[ :, v ][ :, 0 ]
			Yt = target_trajectory[ :, v ][ :, 1 ]
			Zt = target_trajectory[ :, v ][ :, 2 ]

			view.plot( Xt, Yt, Zt, ':' )

		X = previous_actual_trajectory[ frame, v ][ 0 ]
		Y = previous_actual_trajectory[ frame, v ][ 1 ]
		Z = previous_actual_trajectory[ frame, v ][ 2 ]

		view.scatter( X, Y, Z )
		inset_view_xz.scatter( X, Z )
		inset_view_yz.scatter( Y, Z )
		inset_view_xy.scatter( X, Y )

	for p1, p2 in positions_pairs.items():
		position1 = deepcopy( previous_actual_trajectory[ frame, positions[ p1 + '_position' ] ] )
		position2 = deepcopy( previous_actual_trajectory[ frame, positions[ p2 + '_position' ] ] )
		position1[ 1: ] *= -1
		position2[ 1: ] *= -1

		try:
			cat12, _, _, _ = get_coor_marker_points_ideal_catenary( *position1, *position2, 3., .05 )
			cat12[ :, 1: ] *= -1
		except:
			cat12 = array( [ position1, position2 ] )

		view.plot( cat12[ :, 0 ], cat12[ :, 1 ], cat12[ :, 2 ] )
		inset_view_xz.plot( cat12[ :, 0 ], cat12[ :, 2 ] )
		inset_view_yz.plot( cat12[ :, 1 ], cat12[ :, 2 ] )
		inset_view_xy.plot( cat12[ :, 0 ], cat12[ :, 1 ] )

	view.axis( 'equal' )
	inset_view_xy.axis( 'equal' )
	inset_view_xz.axis( 'equal' )
	inset_view_yz.axis( 'equal' )
	view.invert_yaxis()
	view.invert_zaxis()

	plt.savefig( f'{folder}/plots/3d_frames/{frame}.png', dpi = 100 )
	plt.close( 'all' )
	del fig, view, inset_view_xz, inset_view_yz, inset_view_xy

100%|██████████| 35/35 [00:17<00:00,  2.05it/s]


In [160]:
collect()

492345

In [161]:
gif_from_pngs( f'{folder}/plots/3d_frames')

In [162]:
system( f'ffmpeg -y -itsscale 0.25 -i {folder}/plots/3d_frames/animation.gif {folder}/plots/animation.mp4' )

0